In [1]:
import sys  
sys.path.insert(0, '/home/iftekhar/myworkplace/AI-system/retrieval_Model/Page_Ranking_Experiment/methods_collection/')
import ranking_help_methods 
import make_question as question_maker
import vectorization
import sequence_handler
import corpus_handling_methods
import pandas as pd
import ranking
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()
TFIDF_vectorizer = TfidfVectorizer()

In [2]:
dataset = pd.read_csv("../../../Helpers/Title_link_merged_corpus.csv")
dataset = dataset.iloc[:,2:]
dataset = dataset.rename(columns={"Article": "Data"})
dataset.Data = dataset.Data.apply(lambda x: corpus_handling_methods.clean_text(x))
dataset.head()

,Data,Summary,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,MobiControl v14 Manual – MobiControl v14 Manual,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,Apple製品 – MobiControl v14 Manual ios overall,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,Android端末 – MobiControl v14 Manual plus overall,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,Windows PC – MobiControl v14 Manual wpc overall,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,Windows Embedded – MobiControl v14 Manual wm o...,4


In [3]:
split_corpus = corpus_handling_methods.corpus_split(dataset, 8)
split_corpus.head()

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,設定 順序 MobiControl 設定 順序 行い 端末 OS,0
2,対象 オンプレミス MobiControl サーバ のみ 必要 な 設定,0
3,事項 iOS 端末 macOS コンピュータ 必須 接続 プロファイル,0
4,作成 ADDS Active Directory Domain Service サーバ 間,0


In [4]:
perpage_dataset = corpus_handling_methods.corpus_per_page(split_corpus)
perpage_dataset.head() 

,Data,PageID
0,カスタマイザー から サブタイトル 設定 MobiControl 設定 順序 MobiCon...,0
1,MobiControl v Manual Apple 製品 MobiControl 管理 i...,1
2,MobiControl v Manual Android 設定 モード A MobiCont...,2
3,MobiControl v Manual Windows PC 根本 セキュリティ 対策 リ...,3
4,MobiControl v Manual Windows Embedded 端末 Windo...,4


In [14]:
sample_size = 10
questions_samples = question_maker.question_dataframe_generator_1000(split_corpus, sample_size)
questions_samples.head()

,Question,PageID
0,グループ 端末 割り当て 完了 割り当て 作成 済み 使うには,202
1,構成 プロファイル 適用 停止 プロファイル 失効 これから 開発を開始する場所,324
2,階 項目 表示 c d つ 項目 どの,32
3,選択 利用 規約 書 登録 編集 管理 使いたいです,127
4,ファイル 位置 データ 表示 始める方法,112


In [15]:
vec = vectorization.vector_fit(TFIDF_vectorizer, split_corpus)

In [16]:
sample_count = 0
sum_score = 0
container = []

for index, col in questions_samples.iterrows():
    print(col['Question'], col['PageID'])
    query = str(col['Question'])
    question_parts = question_maker.making_query_collection(query)
    collector = sequence_handler.sequence_searcher(perpage_dataset, question_parts)
    perpage_sequence_match = sequence_handler.perpage_sequence_match(collector, 
                                        perpage_dataset, split_corpus, query)
    ranks = ranking.crude_ranks(perpage_sequence_match, query, vec)
    #filtered_ranks = ranking.filtering_ranks(ranks, perpage_sequence_match, query, vec)
    
    page_answers = []
    prediction_scores = []
    for ids, score in ranks:
        page_answers.append(ids)
        prediction_scores.append(score)
    MRR_score = ranking.mean_reciprocal_rank_score(col['PageID'], page_answers)
    sum_score += MRR_score
    container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
    sample_count += 1
    
result = pd.DataFrame(container, columns=['score', 'actual_answer',
'page_answers', 'prediction_scores', 'query'])
result.to_csv('seq_matcher_TFIDF_Vectorizer_performance.csv')
score = sum_score/sample_count
score


グループ 端末 割り当て 完了 割り当て 作成 済み 使うには 202
構成 プロファイル 適用 停止 プロファイル 失効 これから 開発を開始する場所 324
階 項目 表示 c d つ 項目 どの 32
選択 利用 規約 書 登録 編集 管理 使いたいです 127
ファイル 位置 データ 表示 始める方法 112
展開 チェックイン サーバ 待ち 行列 溜まっ いる なにで 17
ADFS サーバ による 認証 SSO Sign On 構築するにはどうすればよいですか 229
選択 た VPN グループ 入力 共有 鍵 何が 283
プロファイル 配布 対象 として 指定 結果 として どうですか 324
た 曜日 日時 インストール 追加 ボタン 押す 何ですか 53
含む アーキテクチャ サポート しかし libSOTIMCAgentSDK a ライブラリ 使い始める方法を教えてください 374
ログ 記録 れ WIP によって 不適切 な 構築するにはどうすればよいですか 58


0.7916666666666666

In [13]:
perpage_sequence_match

[[8,
  ['a b 共 適用 ませ ん セキュリティ 対策',
   'a b でも 適用 可能 但し iOS 端末',
   'b 適用 ませ ん macOS コンピュータ a b',
   '端末 移動 a b でも 適用 可能 但し']],
 [10, ['お客様 ページ 作業 実施 いただき SaaS MobiControl 利用']],
 [17, ['b 端末 グループ 指し 端末 登録 名前 登録', 'グループ a 端末 グループ b 端末 グループ 指し']],
 [18, ['アドレス a b 端末 登録 作成 生成 れ', 'a b 運用 実施 完全 閉域 網 Android']],
 [22, ['SOTI Hub 端末 インストール た だけ イントラネットゲートウェイ 経由']],
 [29, ['説明 a b c クリック 説明 開い た']],
 [61, ['ジャンル a c 混合 ジャンル a b c', 'ソフト マル ウェア 判定 ドライバ a b ない']],
 [68, ['現われ 不 特定 ネットワーク a b g n', '特定 ネットワーク a b g n 接続 モード']],
 [113, ['img src MCExeIcon br MCDisp a br 使っ', 'a br 使っ 改行 から 文字 表示 a']],
 [146, ['a b c アカウント 入力 画面 現れ MobiControl']],
 [158, ['a br 使っ 改行 から 文字 表示 a', 'br MCDisp a br 使っ 改行 から 文字']],
 [217, ['長押 表示 接続 a b 端末 スリープ 状態']],
 [227, ['A B つ 作業 実施 表示 デバイス エージェント']],
 [229, ['サーバ 連携 マッピング ADDS グループ 指定 a b', 'グループ マッピング ADDS ADFS による 認証 a b']],
 [230, ['アプリ Android Plus 端末 a b c 項', 'managed Google Play アクセス a b 設定 できる']],
 [236, ['アカウント 和訳 managed Google Play アカウント a b']],
 [2